In [1]:
import cv2
import numpy as np
from sklearn import cluster

In [9]:
params = cv2.SimpleBlobDetector_Params()

params.filterByInertia
params.minInertiaRatio = 0.6

detector = cv2.SimpleBlobDetector_create(params)

def get_blobs(frame):
    frame_blurred = cv2.medianBlur(frame, 7)
    frame_gray = cv2.cvtColor(frame_blurred, cv2.COLOR_BGR2GRAY)
    blobs = detector.detect(frame_gray)
    
    return blobs


In [12]:
def get_dice_from_blobs(blobs):
    # get centroids of all blobs
    X = []
    for b in blobs:
        pos = b.pt
        
        if pos != None:
            X.append(pos)
    X = np.asarray(X)
    
    if len(X) > 0:
        # Important to set min_sample to 0, as a dice may only have one dot
        clustering = cluster.DBSCAN(eps=40, min_samples=1).fit(X)
        
        # Find the largest label assigned + 1, to get the number of dice found
        num_dice = max(clustering.labels_) + 1
        
        dice = []
        
        # Calculate centroid of each dice, the average between all dice's dots
        for i in range(num_dice):
            X_dice = X[clustering.labels_ == i]
            
            centroid_dice = np.mean(X_dice, axis=0)
            
            dice.append([len(X_dice), *centroid_dice])
        
        return dice
    else:
        return []

In [14]:
def overlay_info(frame, dice, blobs):
    # Overlay blobs
    for b in blobs:
        pos = b.pt
        r = b.size / 2
        
        cv2.circle(frame, (int(pos[0]), int(pos[1])),
                   int(r), (255, 0, 0), 2)
        # overlay dice number
        for d in dice:
            # get textsize for text centering
            text_size = cv2.getTextSize(str(d[0]), cv2.FONT_HERSHEY_PLAIN, 3, 2)[0]
            
            cv2.putText(frame, str(d[0]),
                        (int(d[1] - text_size[0] / 2),
                         int(d[2] + text_size[1] / 2)),
                         cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 2)                      

In [15]:
# Initialize a video feed
cap = cv2.VideoCapture(0)

while True:
    # Get the latest image from the video feed 
    ret, frame = cap.read()
    
    blobs = get_blobs(frame)
    dice = get_dice_from_blobs(blobs)
    out_frame = overlay_info(frame, dice, blobs)
    
    cv2.imshow("frame", frame)
    
    res = cv2.waitKey(1)
    
    # Stop if the user presses "q"
    if res & 0xFF == ord("q"):
        break

# Release the capture when everything is done
cap.release()
cv2.destroyAllWindows()